In [0]:
%tensorflow_version 2.x

In [21]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, \
  concatenate, Dense, Activation, Dropout, GlobalMaxPooling1D, Flatten

2.1.0-rc1


In [0]:
# Parameters:
# Word Embedding
max_features = 1000
maxlen = 400
embedding_dims = 100

# Convolution
kernel_sizes = [3, 4, 5]  # or filter length
filters = 128
pool_size = 4 ## MaxPooling vs GlobalMaxPooling

# Training
batch_size = 30
epochs = 2

In [9]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [0]:
# Network Architecture
# Deeper Multi-branch 
var_input = Input(shape=(400,))
emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)
#text_embed = SpatialDropout1D(0.2)(emb_layer)
conv_branches = []
for kernel_size in kernel_sizes:
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='relu',
                 strides=1)(emb_layer)
  # Batch Normalization or Dropout?
  conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='relu',
                 strides=1)(conv_layer_1)
  # Batch Normalization or Dropout?
  maxpooling = GlobalMaxPooling1D()(conv_layer_2)
  conv_branches.append(maxpooling)

merge_layer = concatenate(conv_branches)
dense_layer = Dense(1, activation='sigmoid')(merge_layer)
model = Model(inputs=var_input, outputs=dense_layer)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 23s 904us/sample - loss: 0.4158 - accuracy: 0.7974 - val_loss: 0.3183 - val_accuracy: 0.8614
Epoch 2/2
25000/25000 [==============================] - 16s 622us/sample - loss: 0.2734 - accuracy: 0.8854 - val_loss: 0.2798 - val_accuracy: 0.8821


In [14]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 4s 168us/sample - loss: 0.2798 - accuracy: 0.8821
Test score: 0.2798379421412945
Test accuracy: 0.88208


In [0]:
# Network Architecture
# Deeper Multi-branch + SELU
var_input = Input(shape=(400,))
emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)
#text_embed = SpatialDropout1D(0.2)(emb_layer)
conv_branches = []
for kernel_size in kernel_sizes:
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='selu',
                 strides=1)(emb_layer)
  # Batch Normalization or Dropout?
  conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='selu',
                 strides=1)(conv_layer_1)
  # Batch Normalization or Dropout?
  maxpooling = GlobalMaxPooling1D()(conv_layer_2)
  conv_branches.append(maxpooling)

merge_layer = concatenate(conv_branches)
dense_layer = Dense(1, activation='sigmoid')(merge_layer)
model = Model(inputs=var_input, outputs=dense_layer)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 400)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 400, 100)     100000      input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 398, 128)     38528       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 397, 128)     51328       embedding_3[0][0]                
____________________________________________________________________________________________

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 17s 664us/sample - loss: 0.3879 - accuracy: 0.8160 - val_loss: 0.3222 - val_accuracy: 0.8632
Epoch 2/2
25000/25000 [==============================] - 16s 640us/sample - loss: 0.2626 - accuracy: 0.8895 - val_loss: 0.2813 - val_accuracy: 0.8809


In [17]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 4s 172us/sample - loss: 0.2813 - accuracy: 0.8809
Test score: 0.281250522659719
Test accuracy: 0.88092


In [22]:
# Network Architecture
# Deeper Multi-branch w/o globalmaxpooling
var_input = Input(shape=(400,))
emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)
#text_embed = SpatialDropout1D(0.2)(emb_layer)
conv_branches = []
for kernel_size in kernel_sizes:
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='selu',
                 strides=1)(emb_layer)
  # Batch Normalization or Dropout?
  conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='selu',
                 strides=1)(conv_layer_1)
  # Batch Normalization or Dropout?
  maxpooling = MaxPooling1D(pool_size=3)(conv_layer_2)
  flattened = Flatten()(maxpooling)
  conv_branches.append(flattened)

merge_layer = concatenate(conv_branches)
dense_layer = Dense(1, activation='sigmoid')(merge_layer)
model = Model(inputs=var_input, outputs=dense_layer)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 400)]        0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 400, 100)     100000      input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 398, 128)     38528       embedding_6[0][0]                
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 397, 128)     51328       embedding_6[0][0]                
____________________________________________________________________________________________

In [23]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 17s 679us/sample - loss: 0.4333 - accuracy: 0.7890 - val_loss: 0.3558 - val_accuracy: 0.8458
Epoch 2/2
25000/25000 [==============================] - 16s 641us/sample - loss: 0.3250 - accuracy: 0.8661 - val_loss: 0.5017 - val_accuracy: 0.8193
